# Solución estática con función $F(r)$ 

Nota: Este cuaderno fue hecho usando SageMath 9.1

In [1]:
version()

'SageMath version 9.1, Release Date: 2020-05-20'

Se activa la visualización matemática de latex

In [2]:
%display latex

Se usó paralelismo usando 10 hilos

In [3]:
Parallelism().set(nproc=10)

# 1. Definición del espacio tiempo y variables

El espacio-tiempo se define como una variedad lorentziana de 4 dimensiones.

In [4]:
M=Manifold(4, 'M', latex_name=r"\mathcal{M}", structure="Lorentzian")

Luego, se introducen las coordenadas esféricas usando el método ```.chart()``` del objeto M. Este método recibe una cadena de texto que incluye el nombre de la coordenada, su rango y el símbolo de Latex con el que se representará. Si no se pasa rango se asumirá como $(-\infty,+\infty)$, si se omite el símbolo de latex, la representación será con el nombre de la coordenada.

In [5]:
BL.<t,r,theta,phi> = M.chart(r"t r:(0,+oo) theta:(0,pi):\theta phi:(0,2*pi):\phi")

Se definen las variables que usaremos a lo largo del código, usando la función ```var()```.
Adicionalmente se define una función $F(r)$ usando la función ```function()```.

In [6]:
var("m, Lambda, gamma, zeta, c, alpha, beta, m_g", domain="real")
F=function("F",nargs=1)(r)
assume(m>=0)
assume(m_g>=0)

# 2. Tensor métrico

Para encontrar la función $F(r)$, con la cual se puede escribir el elemento de linea como sigue

\begin{equation}
    ds^2=-F(r)dt^2+\dfrac{1}{F(r)}dr^2+r^2d\Omega^2
\end{equation}

y que satisface las ecuaciones de campo.

Dado que el elemento de linea se puede escribir como
\begin{equation}
    ds^2=g_{\mu\nu}dx^\mu dx^\nu
\end{equation}
el tensor métrico $g_{\mu\nu}$  queda como

In [7]:
g_f=M.metric()
g_f[0,0]=-F
g_f[1,1]=1/F
g_f[2,2]=r^2
g_f[3,3]=(r*sin(theta))^2
g_f_ud=g_f.up(g_f,0)
g_f[:]

[           -F(r)                0                0                0]
[               0           1/F(r)                0                0]
[               0                0              r^2                0]
[               0                0                0 r^2*sin(theta)^2]

# 3. Tensor de Einstein

## 3.1 Tensor de Ricci $R_{\mu\nu}$

El tensor de Ricci, usando el método ```.ricci()```, para la métrica es

In [8]:
Ricci_f=g_f.ricci()
Ricci_f[:]

[1/2*(r*F(r)*d^2(F)/dr^2 + 2*F(r)*d(F)/dr)/r                                           0                                           0                                           0]
[                                          0   -1/2*(r*d^2(F)/dr^2 + 2*d(F)/dr)/(r*F(r))                                           0                                           0]
[                                          0                                           0                       -r*d(F)/dr - F(r) + 1                                           0]
[                                          0                                           0                                           0        -(r*d(F)/dr + F(r) - 1)*sin(theta)^2]

## 3.2 Escalar de curvatura de Ricci $R$

El escalar de curvatura de Ricci, contrayendo la métrica con el tensor de Ricci, usando el método ```.ricci_scalar()``` queda como

In [9]:
R_f=g_f.ricci_scalar()
R_f.expr()

-(r^2*diff(F(r), r, r) + 4*r*diff(F(r), r) + 2*F(r) - 2)/r^2

## 3.3 Tensor Einstein $G^\mu_\nu$

Ahora definimos el Tensor de Einstein, y contruimos las componentes mixtas.

In [10]:
G_f_ud=Ricci_f-R_f*g_f/2
G_f_ud=G_f_ud.up(g_f,0)
G_f_ud[:]

[       (r*d(F)/dr + F(r) - 1)/r^2                                 0                                 0                                 0]
[                                0        (r*d(F)/dr + F(r) - 1)/r^2                                 0                                 0]
[                                0                                 0 1/2*(r*d^2(F)/dr^2 + 2*d(F)/dr)/r                                 0]
[                                0                                 0                                 0 1/2*(r*d^2(F)/dr^2 + 2*d(F)/dr)/r]

# 4. Tensor de momento energía efectivo de la teoría $X_\nu^\mu$

Usando la definición del tensor $\mathcal{K}^\mu_\nu$ 

\begin{equation}
    \mathcal{K}^\mu_\mu=1-\sqrt{g^{\mu\mu}f_{\mu\mu}}
\end{equation}

con $f=diag(0,0,c^2,c^2\sin^2\theta)$. La expresión anterior se obtiene usando que tanto $g$ como $f$ son tensores diagonales, es decir, los elementos fuera de la diagonal de $\mathcal{K}^\mu_\nu$ son cero. Luego, el tensor es

In [11]:
K_f=M.tensor_field(1, 1, name='K1') 
K_f[0,0]=1
K_f[1,1]=K_f[0,0]
K_f[2,2]=1-c/r
K_f[3,3]=1-c/(r)
K_f[:]

[       1        0        0        0]
[       0        1        0        0]
[       0        0 -c/r + 1        0]
[       0        0        0 -c/r + 1]

## 4.1 $\mathcal{K}^2$ y $\mathcal{K}^3$

El tensor $\mathcal{K}^{2\mu}_\nu=\mathcal{K}^{\mu}_\sigma\mathcal{K}^{\sigma}_\nu$ será entonces

In [12]:
K2_f=K_f.contract(K_f)
K2_f[:]

[                      1                       0                       0                       0]
[                      0                       1                       0                       0]
[                      0                       0 (c^2 - 2*c*r + r^2)/r^2                       0]
[                      0                       0                       0 (c^2 - 2*c*r + r^2)/r^2]

El tensor $\mathcal{K}^{3\mu}_\nu=\mathcal{K}^{\mu}_\sigma\mathcal{K}^{\sigma}_\rho\mathcal{K}^{\rho}_\nu$ será entonces

In [13]:
K3_f=K_f.contract(K_f).contract(K_f)
K3_f[:]

[                                   1                                    0                                    0                                    0]
[                                   0                                    1                                    0                                    0]
[                                   0                                    0 -(c^3 - 3*c^2*r + 3*c*r^2 - r^3)/r^3                                    0]
[                                   0                                    0                                    0 -(c^3 - 3*c^2*r + 3*c*r^2 - r^3)/r^3]

## 4.2 Trazas y potenciales

Primero, se calculan las trazas de cada una de las matrices calculadas anteriormente

In [14]:
K=K_f.trace()
K2=K2_f.trace()
K3=K3_f.trace()

Luego, se calculan los potenciales $U_2$ y $U_3$.

In [15]:
U2=K^2-K2
U2.expr()

2*(c^2 - 6*c*r + 6*r^2)/r^2

In [16]:
U3=K^3-3*K*K2+2*K3
U3.expr()

12*(c^2 - 3*c*r + 2*r^2)/r^2

## 4.3 Tensor momento energía efectivo

El tensor $X^{\mu}_\nu$ viene dado por

\begin{equation}
    X^\mu_\nu=\mathcal{K}^\mu_{\nu}-[\mathcal{K}]g^\mu_\nu-\alpha\left(\mathcal{K}^{2\mu}_{\nu}-[\mathcal{K}]\mathcal{K}^\mu_\nu+\dfrac{U_2}{2}g^\mu_{\nu}\right)+3\beta\left(\mathcal{K}^{3\mu}_\nu-[\mathcal{K}]\mathcal{K}^{2\mu}_\nu+\dfrac{U_2}{2}\mathcal{K}^\mu_\nu-\dfrac{U_3}{6}g^\mu_{\nu}\right).
\end{equation}

Luego, este queda como

In [17]:
X_ud=K_f-K*g_f_ud-alpha*(K2_f-K*K_f+U2*g_f_ud/2)+3*beta*(K3_f-K*K2_f+K_f*U2/2-g_f_ud*U3/6)
X_ud[:]

[-((alpha + 3*beta)*c^2 - 2*(2*alpha + 3*beta + 1)*c*r + 3*(alpha + beta + 1)*r^2)/r^2                                                                                     0                                                                                     0                                                                                     0]
[                                                                                    0 -((alpha + 3*beta)*c^2 - 2*(2*alpha + 3*beta + 1)*c*r + 3*(alpha + beta + 1)*r^2)/r^2                                                                                     0                                                                                     0]
[                                                                                    0                                                                                     0                                 ((2*alpha + 3*beta + 1)*c - 3*(alpha + beta + 1)*r)/r                                                                                     0]
[                                                                                    0                                                                                     0                                                                                     0                                 ((2*alpha + 3*beta + 1)*c - 3*(alpha + beta + 1)*r)/r]

# 5. Ecuaciones de campo y solución

Las ecuaciones de campo de la teoría de gravedad másiva en el vació, vienen dadas por

\begin{equation}
    G^\mu_\nu=-m_g^2X^\mu_\nu,
\end{equation}

Tomando la componente temporal de la ecuación se obtiene la siguiente ecuación diferencial.

\begin{equation}
    \dfrac{1}{r}\dfrac{dF(r)}{dr}+\dfrac{F}{r^2}-\dfrac{1}{r^2}=m_g^2\dfrac{(\alpha+3\beta)c^2-2(2\alpha+3\beta+1)cr+3(\alpha+\beta+1)r^2}{r^2}.
\end{equation}


A continuación se define la ecuación diferencial

In [18]:
diffEq=diff(F,r)/r+F/r^2-1/r^2==m_g^2*((alpha+3*beta)*c^2-2*(2*alpha+3*beta+1)*c*r+3*(alpha+beta+1)*r^2)/r^2

Luego, usando la función ```desolve()```, se resuelve la ecuación diferencial anterior

In [ ]:
desolve(diffEq,F,ivar=r)

((alpha + 3*beta)*c^2*m_g^2*r - (2*alpha + 3*beta + 1)*c*m_g^2*r^2 + (alpha + beta + 1)*m_g^2*r^3 + _C + r)/r

donde, si se define
\begin{gather}
    \Lambda = 3m_g^2(\alpha+\beta+1), \\
    \gamma = -m_g^2c(2\alpha+3\beta+1),\\
    \zeta = m_g^2c^2(\alpha+3\beta),
\end{gather}

y se ajusta la constante de integración $C$ como $2M$, se obtiene

\begin{equation}
    F(r)=1-\dfrac{2M}{r}+\dfrac{\Lambda}{3}r^2+\gamma r +\zeta.
\end{equation}